In [80]:
# Import packages
import pandas as pd
from sqlalchemy import create_engine
import json
import os

pd.set_option('display.max_rows', 200)

In [81]:
%load_ext dotenv
%dotenv ../.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [82]:
con = create_engine(os.getenv('EDM_DATA'))

In [83]:
# Build funtion to get the domains associated with a specific field
def get_domain(field, domain_path):
    with open(domain_path, "r") as domain:
        domain = json.load(domain)
    records = [i for i in domain['fields'] if i['field'] == field]
    if len(records) == 0: 
        print(f'cannot find field: {field} in domain table')
    else: 
        return records[0]['domain']

In [84]:
# Example
proxcode_domain = get_domain('proxcode', 'pluto_domains.json')
print(proxcode_domain)

{'0': 'Not Available', '1': 'Detached', '2': 'Semi-Attached', '3': 'Attached'}


In [85]:
# Example
proxcode_domain.keys()

dict_keys(['0', '1', '2', '3'])

In [86]:
# Specify latest PLUTO version
pluto = 'dcp_pluto."19v2_w_corrections"'
# Specify previous PLUTO version or the one you want to compare to
plutoprev = 'dcp_pluto."19v2_wo_corrections"'

### Value check for the thirteen fields with expected domain values

##### includes summary statistics of unique value comparisions for the two PLUTO version, and lists of values that do not agree w/ domain list

### Zoning districts

In [87]:
sql = f'''
    WITH newcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {pluto})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist),
    prevcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {plutoprev})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist)
    SELECT a.zonedist, b.zonedist as zonedistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.zonedist = b.zonedist
    ORDER BY diff DESC;
'''

In [88]:
df = pd.read_sql(sql=sql, con=con)

In [89]:
df[df['diff'] != 0]

,zonedist,zonedistprev,countnew,countprev,diff
0,None,None,2561097.0,NaN,NaN
1,None,None,NaN,2561097.0,NaN


### Commercial overlay

In [90]:
sql = f'''
    WITH newcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {pluto}
    UNION ALL
    SELECT overlay2 as overlay FROM {pluto}
    )
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay),
    prevcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {plutoprev}
    UNION ALL
    SELECT overlay2 as overlay FROM {plutoprev})
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay)
    SELECT a.overlay, b.overlay as overlayprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.overlay = b.overlay
    ORDER BY a.overlay;
'''

In [91]:
df = pd.read_sql(sql=sql, con=con)

In [92]:
df

,overlay,overlayprev,countnew,countprev,diff
0,C1-1,C1-1,1587.0,1587.0,0.0
1,C1-2,C1-2,11031.0,11031.0,0.0
2,C1-3,C1-3,13790.0,13790.0,0.0
3,C1-4,C1-4,8097.0,8097.0,0.0
4,C1-5,C1-5,2541.0,2541.0,0.0
5,C2-1,C2-1,1684.0,1684.0,0.0
6,C2-2,C2-2,6023.0,6023.0,0.0
7,C2-3,C2-3,11949.0,11949.0,0.0
8,C2-4,C2-4,16721.0,16721.0,0.0
9,C2-5,C2-5,1523.0,1523.0,0.0


### Special purpose district

In [93]:
sql = f'''
    WITH newcount as (
    WITH sdall as (
    SELECT spdist1 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist2 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist3 as spdist FROM {pluto}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist),
    prevcount as (
     WITH sdall as (
    SELECT spdist1 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist2 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist3 as spdist FROM {plutoprev}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist)
    SELECT a.spdist, b.spdist as spdistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.spdist = b.spdist
    ORDER BY diff DESC;
'''

In [94]:
df = pd.read_sql(sql=sql, con=con)

In [95]:
df[df['diff'] != 0]

,spdist,spdistprev,countnew,countprev,diff
0,None,None,2476702.0,NaN,NaN
1,None,None,NaN,2476702.0,NaN


In [96]:
#Check spdist domain
unique_spdist = df.spdist.tolist()
spdist_domain = list(get_domain('spdist1', 'pluto_domains.json').keys())

In [97]:
# in domain not in pluto:
[i for i in spdist_domain if i not in unique_spdist]

['MX-3']

In [98]:
# in pluto not in domain:
[i for i in unique_spdist if i not in spdist_domain]

[None, None, 'MX-20', 'MX-16/EC-5', 'BSC', 'SW', 'MX-16/EC-6']

### Limited height

In [99]:
sql = f'''
    WITH newcount as (
    SELECT ltdheight, count(*)
    FROM {pluto}
    GROUP BY ltdheight),
    prevcount as (
    SELECT ltdheight, count(*)
    FROM {plutoprev}
    GROUP BY ltdheight)
    SELECT a.ltdheight, b.ltdheight as ltdheightprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ltdheight = b.ltdheight
    ORDER BY a.ltdheight;
'''

In [100]:
df = pd.read_sql(sql=sql, con=con)

In [101]:
df

,ltdheight,ltdheightprev,countnew,countprev,diff
0,LH-1,LH-1,2239.0,2239.0,0.0
1,LH-1A,LH-1A,797.0,797.0,0.0
2,None,None,856459.0,NaN,NaN
3,None,None,NaN,856459.0,NaN


In [102]:
#Check ltdheight domain
unique_ltdheight = df.ltdheight.tolist()
ltdheight_domain = list(get_domain('ltdheight', 'pluto_domains.json').keys())

In [103]:
# in domain not in pluto:
[i for i in ltdheight_domain if i not in unique_ltdheight]

['LH-2', 'LH-3']

In [104]:
# in pluto not in domain:
[i for i in unique_ltdheight if i not in ltdheight_domain]

[None, None]

### Building class

In [105]:
sql = f'''
    WITH newcount as (
    SELECT bldgclass, count(*)
    FROM {pluto}
    GROUP BY bldgclass),
    prevcount as (
    SELECT bldgclass, count(*)
    FROM {plutoprev}
    GROUP BY bldgclass)
    SELECT a.bldgclass, b.bldgclass as bldgclassprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bldgclass = b.bldgclass
    ORDER BY diff DESC;
'''

In [106]:
df = pd.read_sql(sql=sql, con=con)

In [107]:
df[df['diff'] != 0]

,bldgclass,bldgclassprev,countnew,countprev,diff
0,None,None,NaN,509.0,NaN
1,None,None,509.0,NaN,NaN


In [108]:
#Check bldgclass domain
unique_bldgclass = df.bldgclass.tolist()
bldgclass_domain = list(get_domain('bldgclass', 'pluto_domains.json').keys())

In [109]:
# in domain not in pluto:
[i for i in bldgclass_domain if i not in unique_bldgclass]

['A',
 'B',
 'C',
 'D',
 'E',
 'E3',
 'E4',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'R7',
 'S',
 'T',
 'U',
 'U3',
 'V',
 'W',
 'Y',
 'Z',
 'Z6']

In [110]:
# in pluto not in domain:
[i for i in unique_bldgclass if i not in bldgclass_domain]

[None, None]

### Land use

In [111]:
sql = f'''
    WITH newcount as (
    SELECT landuse, count(*)
    FROM {pluto}
    GROUP BY landuse),
    prevcount as (
    SELECT landuse, count(*)
    FROM {plutoprev}
    GROUP BY landuse)
    SELECT a.landuse, b.landuse as landuseprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.landuse = b.landuse
    ORDER BY a.landuse;
'''

In [112]:
df = pd.read_sql(sql=sql, con=con)

In [113]:
df[df['diff'] != 0]

,landuse,landuseprev,countnew,countprev,diff
11,None,None,3018.0,NaN,NaN
12,None,None,NaN,3018.0,NaN


In [114]:
#Check landuse domain
unique_landuse = df.landuse.tolist()
landuse_domain = list(get_domain('landuse', 'pluto_domains.json').keys())

In [115]:
# in domain not in pluto:
[i for i in landuse_domain if i not in unique_landuse]

[]

In [116]:
# in pluto not in domain:
[i for i in unique_landuse if i not in landuse_domain]

[None, None]

### Owner type

In [117]:
sql = f'''
    WITH newcount as (
    SELECT ownertype, count(*)
    FROM {pluto}
    GROUP BY ownertype),
    prevcount as (
    SELECT ownertype, count(*)
    FROM {plutoprev}
    GROUP BY ownertype)
    SELECT a.ownertype, b.ownertype as ownertypeprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ownertype = b.ownertype
    ORDER BY a.ownertype;
'''

In [118]:
df = pd.read_sql(sql=sql, con=con)

In [119]:
df

,ownertype,ownertypeprev,countnew,countprev,diff
0,C,C,11654.0,11654.0,0.0
1,M,M,80.0,80.0,0.0
2,O,O,1373.0,1373.0,0.0
3,P,P,522.0,522.0,0.0
4,X,X,17414.0,17414.0,0.0
5,None,None,828452.0,NaN,NaN
6,None,None,NaN,828452.0,NaN


In [120]:
#Check ownertype domain
unique_ownertype = df.ownertype.tolist()
ownertype_domain = list(get_domain('ownertype', 'pluto_domains.json').keys())

In [121]:
# in domain not in pluto:
[i for i in ownertype_domain if i not in unique_ownertype]

['NaN']

In [122]:
# in pluto not in domain:
[i for i in unique_ownertype if i not in ownertype_domain]

[None, None]

### Area source

In [123]:
sql = f'''
    WITH newcount as (
    SELECT areasource, count(*)
    FROM {pluto}
    GROUP BY areasource),
    prevcount as (
    SELECT areasource, count(*)
    FROM {plutoprev}
    GROUP BY areasource)
    SELECT a.areasource, b.areasource as areasourceprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.areasource = b.areasource
    ORDER BY a.areasource;
'''

In [124]:
df = pd.read_sql(sql=sql, con=con)

In [125]:
df

,areasource,areasourceprev,countnew,countprev,diff
0,0,0,139.0,139.0,0.0
1,2,2,813263.0,813263.0,0.0
2,4,4,179.0,179.0,0.0
3,5,5,63.0,63.0,0.0
4,7,7,45342.0,45342.0,0.0
5,None,None,509.0,NaN,NaN
6,None,None,NaN,509.0,NaN


In [126]:
#Check areasource domain
unique_areasource = df.areasource.tolist()
areasource_domain = list(get_domain('areasource', 'pluto_domains.json').keys())

In [127]:
# in domain not in pluto:
[i for i in areasource_domain if i not in unique_areasource]

[]

In [128]:
# in pluto not in domain:
[i for i in unique_areasource if i not in areasource_domain]

[None, None]

### Extension

In [129]:
sql = f'''
    WITH newcount as (
    SELECT ext, count(*)
    FROM {pluto}
    GROUP BY ext),
    prevcount as (
    SELECT ext, count(*)
    FROM {plutoprev}
    GROUP BY ext)
    SELECT a.ext, b.ext as extprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ext = b.ext
    ORDER BY a.ext;
'''

In [130]:
df = pd.read_sql(sql=sql, con=con)

In [131]:
df

,ext,extprev,countnew,countprev,diff
0,E,E,48335.0,48335.0,0.0
1,EG,EG,36709.0,36709.0,0.0
2,G,G,264413.0,264413.0,0.0
3,N,N,451333.0,451333.0,0.0
4,None,None,58705.0,NaN,NaN
5,None,None,NaN,58705.0,NaN


In [132]:
#Check extension domain
unique_extension = df.ext.tolist()
extension_domain = list(get_domain('ext', 'pluto_domains.json').keys())

In [133]:
# in domain not in pluto:
[i for i in extension_domain if i not in unique_extension]

[]

In [134]:
# in pluto not in domain:
[i for i in unique_extension if i not in extension_domain]

['N', None, None]

### Proxy code

In [135]:
sql = f'''
    WITH newcount as (
    SELECT proxcode, count(*)
    FROM {pluto}
    GROUP BY proxcode),
    prevcount as (
    SELECT proxcode, count(*)
    FROM {plutoprev}
    GROUP BY proxcode)
    SELECT a.proxcode, b.proxcode as proxcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.proxcode = b.proxcode
    ORDER BY a.proxcode;
'''

In [136]:
df = pd.read_sql(sql=sql, con=con)

In [137]:
df

,proxcode,proxcodeprev,countnew,countprev,diff
0,0,0,144832.0,144832.0,0.0
1,1,1,313731.0,313731.0,0.0
2,2,2,199995.0,199995.0,0.0
3,3,3,200428.0,200428.0,0.0
4,None,None,509.0,NaN,NaN
5,None,None,NaN,509.0,NaN


In [138]:
#Check proxcode domain
unique_proxcode = df.proxcode.tolist()
proxcode_domain = list(get_domain('proxcode', 'pluto_domains.json').keys())

In [139]:
# in domain not in pluto:
[i for i in proxcode_domain if i not in unique_proxcode]

[]

In [140]:
# in pluto not in domain:
[i for i in unique_proxcode if i not in proxcode_domain]

[None, None]

### Lot type

In [141]:
sql = f'''
    WITH newcount as (
    SELECT lottype, count(*)
    FROM {pluto}
    GROUP BY lottype),
    prevcount as (
    SELECT lottype, count(*)
    FROM {plutoprev}
    GROUP BY lottype)
    SELECT a.lottype, b.lottype as lottypeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.lottype = b.lottype
    ORDER BY a.lottype;
'''

In [142]:
df = pd.read_sql(sql=sql, con=con)

In [143]:
df

,lottype,lottypeprev,countnew,countprev,diff
0,0,0,8534.0,8534.0,0.0
1,1,1,1454.0,1454.0,0.0
2,2,2,1823.0,1823.0,0.0
3,3,3,100674.0,100674.0,0.0
4,4,4,3443.0,3443.0,0.0
5,5,5,738045.0,738045.0,0.0
6,6,6,2228.0,2228.0,0.0
7,7,7,64.0,64.0,0.0
8,8,8,2531.0,2531.0,0.0
9,9,9,190.0,190.0,0.0


In [144]:
#Check lottype domain
unique_lottype = df.lottype.tolist()
lottype_domain = list(get_domain('lottype', 'pluto_domains.json').keys())

In [145]:
# in domain not in pluto:
[i for i in lottype_domain if i not in unique_lottype]

[]

In [146]:
# in pluto not in domain:
[i for i in unique_lottype if i not in lottype_domain]

[None, None]

### Basement code

In [147]:
sql = f'''
    WITH newcount as (
    SELECT bsmtcode, count(*)
    FROM {pluto}
    GROUP BY bsmtcode),
    prevcount as (
    SELECT bsmtcode, count(*)
    FROM {plutoprev}
    GROUP BY bsmtcode)
    SELECT a.bsmtcode, b.bsmtcode as bsmtcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bsmtcode = b.bsmtcode
    ORDER BY a.bsmtcode;
'''

In [148]:
df = pd.read_sql(sql=sql, con=con)

In [149]:
df

,bsmtcode,bsmtcodeprev,countnew,countprev,diff
0,0,0,74507.0,74507.0,0.0
1,1,1,153674.0,153674.0,0.0
2,2,2,495893.0,495893.0,0.0
3,3,3,1499.0,1499.0,0.0
4,4,4,4704.0,4704.0,0.0
5,5,5,128709.0,128709.0,0.0
6,None,None,509.0,NaN,NaN
7,None,None,NaN,509.0,NaN


In [150]:
#Check bsmtcode domain
unique_bsmtcode = df.bsmtcode.tolist()
bsmtcode_domain = list(get_domain('bsmtcode', 'pluto_domains.json').keys())

In [151]:
# in domain not in pluto:
[i for i in bsmtcode_domain if i not in unique_bsmtcode]

[]

In [152]:
# in pluto not in domain:
[i for i in unique_bsmtcode if i not in bsmtcode_domain]

[None, None]

### PLUTO map id

In [153]:
sql = f'''
    WITH newcount as (
    SELECT plutomapid, count(*)
    FROM {pluto}
    GROUP BY plutomapid),
    prevcount as (
    SELECT plutomapid, count(*)
    FROM {plutoprev}
    GROUP BY plutomapid)
    SELECT a.plutomapid, b.plutomapid as plutomapidprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.plutomapid = b.plutomapid
    ORDER BY a.plutomapid;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check plutomapid domain
unique_plutomapid = df.plutomapid.tolist()
plutomapid_domain = list(get_domain('plutomapid', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in plutomapid_domain if i not in unique_plutomapid]

In [ ]:
# in pluto not in domain:
[i for i in unique_plutomapid if i not in plutomapid_domain]

#### done